In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler

import cmdstanpy
from cmdstanpy import CmdStanModel

In [7]:
df = pd.read_csv("df_reduced.csv")
print(df.shape)

(1021, 19)


In [8]:
df.columns

Index(['CAI', 'Trigliceridi', 'Colesterolo_Hdl', 'Glucosio', 'PMAX', 'BMI',
       'Alanina_aminotransferasi_alt', 'Colesterolo_totale',
       'Distribuzione_di_volume', 'Ematocrito_hct', 'Eosinofili_perc',
       'Leucociti_wbc', 'Linfociti_perc', 'Monociti_perc', 'Piastrine',
       'Polso', 'Proteine_totali', 'Volume_medio', 'Eta'],
      dtype='object')

In [9]:
stan_code = r"""
data {
  int<lower=1> N;                       // total observations
  int<lower=1> I;                       // number of donors
  int<lower=1> K;                       // number of target variables (K = 5)
  int<lower=1> P;                       // number of covariates

  array[N] int<lower=1, upper=I> id;     // donor index for each observation
  matrix[N, K] Y;                       // log target variables (preferably centered)
  matrix[N, P] X;                       // covariate matrix
}

parameters {
  // Fixed effects
  matrix[P, K] beta;

  // Random effects
  // standard normal latent variables
  matrix[I, K] z_b;

  // deviations (sd) of random effects
  vector<lower=0>[K] tau_b;

  // correlation between components of random effects
  cholesky_factor_corr[K] L_Omega_b;

  // Residual covariance
  // residual standard deviations
  vector<lower=0>[K] tau_eps;

  // residual correlation
  cholesky_factor_corr[K] L_Omega_eps;
}

transformed parameters {
  // Cholesky of covariance for random effects
  matrix[K, K] L_Sigma_b = diag_pre_multiply(tau_b, L_Omega_b);

  // random effects non-centered: b_i = L_Sigma_b * z_b_i
  matrix[I, K] b;
  {
    matrix[K, I] z_b_t = z_b';          // (K x I)
    matrix[K, I] b_t   = L_Sigma_b * z_b_t;    // (K x I)
    b = b_t';                           // (I x K)
  }

  // Cholesky of residual covariance
  matrix[K, K] L_Sigma = diag_pre_multiply(tau_eps, L_Omega_eps);
}

model {
  // Prior for fixed effects
  to_vector(beta) ~ normal(0, 2);

  // Priors for random effects hyperparameters
  tau_b ~ normal(0, 0.5) T[0, ];      // sd of random effects
  L_Omega_b ~ lkj_corr_cholesky(4);    // moderate prior on correlation
  to_vector(z_b) ~ normal(0, 1);      // non-centered

  // Priors for residual covariance
  tau_eps ~ normal(0, 0.5) T[0, ];      // sd of residuals
  L_Omega_eps ~ lkj_corr_cholesky(4);

  // Likelihood
  for (n in 1:N) {
    row_vector[K] mu_n = X[n] * beta + b[id[n]];
    Y[n] ~ multi_normal_cholesky(mu_n, L_Sigma);
  }
}

generated quantities {
  vector[N] log_lik;

  matrix[N, K] Y_rep;

  // Mean prediction for each observation (N x K)
  matrix[N, K] mu;

  // Full covariance matrices (K x K) from Cholesky factors
  cov_matrix[K] Sigma_b = tcrossprod(L_Sigma_b);
  cov_matrix[K] Sigma_eps = tcrossprod(L_Sigma);

  // Correlation matrices (K x K) from Cholesky factors
  corr_matrix[K] Omega_b = tcrossprod(L_Omega_b);
  corr_matrix[K] Omega_eps = tcrossprod(L_Omega_eps);

  for (n in 1:N) {
    row_vector[K] mu_n_row = X[n] * beta + b[id[n]];

    vector[K] mu_n_vec = mu_n_row';

    mu[n] = mu_n_row;

    // Pointwise Log-likelihood
    log_lik[n] = multi_normal_cholesky_lpdf(Y[n]' | mu_n_vec, L_Sigma);

    // Posterior Prediction
    Y_rep[n] = multi_normal_cholesky_rng(mu_n_vec, L_Sigma)';
  }
}
"""
stan_file = "Model_5.stan"
with open(stan_file, "w") as f:
    f.write(stan_code)

print("Stan model written to:", stan_file)

model = CmdStanModel(stan_file=stan_file)
print("Model compiled.")

11:45:10 - cmdstanpy - INFO - compiling stan file /Users/eli/Desktop/BS_project/Model_5.stan to exe file /Users/eli/Desktop/BS_project/Model_5


Stan model written to: Model_5.stan


11:45:23 - cmdstanpy - INFO - compiled model executable: /Users/eli/Desktop/BS_project/Model_5


Model compiled.


In [10]:
ID_COL = "CAI"
ADD_INTERCEPT = False
target_list = ['PMAX', 'Glucosio', 'Trigliceridi', 'Colesterolo_Hdl', 'BMI']
covariate_cols = df.columns.drop(list(target_list) + [ID_COL])

cols_needed = [ID_COL] + target_list + list(covariate_cols)
df_model = df[cols_needed].dropna().copy()

Y_mat = df_model[target_list].to_numpy(dtype=float)
N, K = Y_mat.shape
X_mat = df_model[covariate_cols].to_numpy(dtype=float)
_, P = X_mat.shape

donor_ids, id_index = np.unique(df_model[ID_COL].to_numpy(), return_inverse=True)
I = len(donor_ids)
id_stan = id_index + 1

stan_data = {
    "N": int(N),
    "I": int(I),
    "K": int(K),
    "P": int(P),
    "Y": Y_mat,
    "X": X_mat,
    "id": id_stan
}

In [11]:
fit = model.sample(
    data=stan_data,
    chains=4,
    parallel_chains=4,
    iter_warmup=1500,
    iter_sampling=1000,
    adapt_delta=0.9,
    max_treedepth=12,
    show_progress=True
)
print(fit.diagnose())

11:45:25 - cmdstanpy - INFO - CmdStan start processing
chain 2:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]

chain 3:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]


chain 4:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]


chain 1:   4%|▊                  | 100/2500 [02:44<1:05:36,  1.64s/it, (Warmup)]

chain 2:   4%|▊                  | 100/2500 [02:54<1:09:59,  1.75s/it, (Warmup)]


chain 2:   8%|█▌                 | 200/2500 [05:50<1:07:09,  1.75s/it, (Warmup)]

chain 3:   8%|█▌                 | 200/2500 [06:08<1:11:56,  1.88s/it, (Warmup)]


chain 2:  12%|██▌                  | 300/2500 [07:58<56:21,  1.54s/it, (Warmup)]

chain 3:  12%|██▎                | 300/2500 [08:26<1:00:31,  1.65s/it, (Warmup)]


chain 1:  16%|███▎                 | 400/2500 [10:06<51:17,  1.47s/it, (Warmup)]

chain 3:  16%|███▎                 | 400/2500 [10:35<52:40,  1.51s/it, (Warmup)]


chain 1:  20%|████▏                | 


12:44:01 - cmdstanpy - INFO - CmdStan done processing.
12:44:01 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_5.stan', line 57, column 2 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[3] is 0, but must

12:44:03 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 3 had 1 iterations at max treedepth (0.1%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


Checking sampler transitions treedepth.
1 of 4000 (0.03%) transitions hit the maximum treedepth limit of 12, or 2^12 leapfrog steps.
Trajectories that are prematurely terminated due to this limit will result in slow exploration.
For optimal performance, increase this limit.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

Rank-normalized split R-hat values satisfactory for all parameters.

Processing complete.



In [12]:
print(fit.diagnose())
summary_df = fit.summary()
summary_df.head(50)

Checking sampler transitions treedepth.
1 of 4000 (0.03%) transitions hit the maximum treedepth limit of 12, or 2^12 leapfrog steps.
Trajectories that are prematurely terminated due to this limit will result in slow exploration.
For optimal performance, increase this limit.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

Rank-normalized split R-hat values satisfactory for all parameters.

Processing complete.



,Mean,MCSE,StdDev,MAD,5%,50%,95%,ESS_bulk,ESS_tail,ESS_bulk/s,R_hat
lp__,10062.700000,0.534746,15.045500,15.361200,10037.400000,10063.100000,10086.900000,798.369,1541.43,0.150732,1.007210
"beta[1,1]",-0.001793,0.000037,0.002743,0.002728,-0.006244,-0.001783,0.002851,5636.280,3249.28,1.064130,0.999794
"beta[1,2]",0.006693,0.000047,0.003614,0.003561,0.000675,0.006646,0.012737,5971.170,3425.31,1.127350,1.000570
"beta[1,3]",0.025227,0.000154,0.012139,0.012059,0.005184,0.025535,0.045247,6327.090,3022.06,1.194550,1.000250
"beta[1,4]",0.000735,0.000055,0.004242,0.004199,-0.006267,0.000744,0.007740,6082.870,3466.93,1.148440,1.000380
"beta[1,5]",0.003557,0.000023,0.001633,0.001644,0.000864,0.003576,0.006236,5283.790,3049.81,0.997577,1.001210
"beta[2,1]",-0.003393,0.000043,0.003096,0.003125,-0.008544,-0.003419,0.001740,5377.610,2693.08,1.015290,1.001670
"beta[2,2]",0.002746,0.000050,0.003934,0.003950,-0.003604,0.002646,0.009215,6064.540,3090.19,1.144980,1.001040
"beta[2,3]",0.099826,0.000187,0.013215,0.013429,0.078192,0.099800,0.121755,5026.230,3613.73,0.948950,1.000140
"beta[2,4]",0.017572,0.000068,0.004742,0.004669,0.009576,0.017664,0.025257,4876.330,3116.60,0.920650,1.000110


In [13]:
with open("Model_5.pkl", "wb") as f:
    pickle.dump(fit, f)